In [5]:
# 📌 Importar conexión
from db_connection import get_connection

# 📌 Conectarse a la base de datos
conexion = get_connection()
cursor = conexion.cursor()


✅ Conexión exitosa a la base de datos


In [6]:
# 📌 Obtener todas las tablas
cursor.execute("SHOW TABLES;")
tablas = [fila[0] for fila in cursor.fetchall()]
print(f"Total de tablas encontradas: {len(tablas)}")


Total de tablas encontradas: 80


In [7]:
tablas_a_borrar = []

for tabla in tablas:
    try:
        cursor.execute(f"SELECT COUNT(*) FROM `{tabla}`;")
        total = cursor.fetchone()[0]
        if total <= 1:
            tablas_a_borrar.append(tabla)
    except Exception as e:
        print(f"❌ Error evaluando tabla {tabla}: {e}")

print(f"\nTablas candidatas para eliminación ({len(tablas_a_borrar)}):")
print(tablas_a_borrar)



Tablas candidatas para eliminación (5):
['coupon_groups', 'electronic_payroll_data', 'electronic_payroll_submissions', 'ledgers', 'payroll_consolidated']


In [8]:
# ⚠️ Solo si tienes respaldo y estás en entorno seguro

# Desactivar restricciones
cursor.execute("SET FOREIGN_KEY_CHECKS=0;")

for tabla in tablas_a_borrar:
    try:
        cursor.execute(f"DROP TABLE `{tabla}`;")
        print(f"✅ Tabla eliminada: {tabla}")
    except Exception as e:
        print(f"❌ Error al eliminar {tabla}: {e}")

# Confirmar cambios
conexion.commit()

# Volver a activar restricciones
cursor.execute("SET FOREIGN_KEY_CHECKS=1;")



✅ Tabla eliminada: coupon_groups
✅ Tabla eliminada: electronic_payroll_data
✅ Tabla eliminada: electronic_payroll_submissions
✅ Tabla eliminada: ledgers
✅ Tabla eliminada: payroll_consolidated
